In [13]:
import pandas as pd
import ijson

In [2]:
file_name = "dataset.json"
with open(file_name, 'r') as f:
    objects = ijson.items(f, 'meta.view.columns.item')
    columns = list(objects)

In [3]:
print columns[0]

{u'name': u'sid', u'format': {}, u'dataTypeName': u'meta_data', u'fieldName': u':sid', u'renderTypeName': u'meta_data', u'position': 0, u'id': -1, u'flags': [u'hidden']}


In [4]:
column_names = [col["fieldName"] for col in columns]

In [5]:
column_names

[u':sid',
 u':id',
 u':position',
 u':created_at',
 u':created_meta',
 u':updated_at',
 u':updated_meta',
 u':meta',
 u'date_of_stop',
 u'time_of_stop',
 u'agency',
 u'subagency',
 u'description',
 u'location',
 u'latitude',
 u'longitude',
 u'accident',
 u'belts',
 u'personal_injury',
 u'property_damage',
 u'fatal',
 u'commercial_license',
 u'hazmat',
 u'commercial_vehicle',
 u'alcohol',
 u'work_zone',
 u'state',
 u'vehicle_type',
 u'year',
 u'make',
 u'model',
 u'color',
 u'violation_type',
 u'charge',
 u'article',
 u'contributed_to_accident',
 u'race',
 u'gender',
 u'driver_city',
 u'driver_state',
 u'dl_state',
 u'arrest_type',
 u'geolocation']

In [6]:
good_columns = [
    "date_of_stop", 
    "time_of_stop", 
    "agency", 
    "subagency",
    "description",
    "location", 
    "latitude", 
    "longitude", 
    "vehicle_type", 
    "year", 
    "make", 
    "model", 
    "color", 
    "violation_type",
    "race", 
    "gender", 
    "driver_state", 
    "driver_city", 
    "dl_state",
    "arrest_type"
]

In [7]:
data = []
with open(file_name, 'r') as f:
    objects = ijson.items(f, 'data.item')
    for row in objects:
        selected_row = []
        for item in good_columns:
            selected_row.append(row[column_names.index(item)])
        data.append(selected_row)

In [8]:
data[0]

[u'2013-09-24T00:00:00',
 u'17:11:00',
 u'MCP',
 u'3rd district, Silver Spring',
 u'DRIVING VEHICLE ON HIGHWAY WITH SUSPENDED REGISTRATION',
 u'8804 FLOWER AVE',
 None,
 None,
 u'02 - Automobile',
 u'2008',
 u'FORD',
 u'4S',
 u'BLACK',
 u'Citation',
 u'BLACK',
 u'M',
 u'MD',
 u'TAKOMA PARK',
 u'MD',
 u'A - Marked Patrol']

In [9]:
dataframe = pd.DataFrame(data, columns=good_columns)

In [13]:
dataframe.date_of_stop=pd.to_datetime(dataframe.date_of_stop)

In [1]:
from pymongo import MongoClient
import json
client = MongoClient()

In [2]:
client = MongoClient('localhost', 27017)

In [3]:
db = client['traffic_violations']
datarec = db.datarec

In [18]:
datarec.insert_many(dataframe.to_dict('records'))

In [8]:
stopsByColor=datarec.aggregate([{"$group" : {"_id":"$color", "count":{"$sum":1}}}])

In [9]:
for doc in stopsByColor:
    print doc

{u'count': 23, u'_id': u'CHROME'}
{u'count': 357, u'_id': u'COPPER'}
{u'count': 4247, u'_id': u'YELLOW'}
{u'count': 2588, u'_id': u'BRONZE'}
{u'count': 5245, u'_id': u'BROWN'}
{u'count': 6491, u'_id': u'GREEN, LGT'}
{u'count': 19330, u'_id': u'MAROON'}
{u'count': 2211, u'_id': u'PURPLE'}
{u'count': 35901, u'_id': u'GOLD'}
{u'count': 167, u'_id': u'PINK'}
{u'count': 746, u'_id': u'CREAM'}
{u'count': 23864, u'_id': u'TAN'}
{u'count': 12812, u'_id': u'GREEN, DK'}
{u'count': 14942, u'_id': u'BLUE, LIGHT'}
{u'count': 14254, u'_id': nan}
{u'count': 43944, u'_id': u'GREEN'}
{u'count': 3829, u'_id': u'ORANGE'}
{u'count': 164915, u'_id': u'WHITE'}
{u'count': 20, u'_id': u'CAMOUFLAGE'}
{u'count': 924, u'_id': u'MULTICOLOR'}
{u'count': 80581, u'_id': u'BLUE'}
{u'count': 199971, u'_id': u'SILVER'}
{u'count': 23227, u'_id': u'BLUE, DARK'}
{u'count': 86613, u'_id': u'RED'}
{u'count': 116635, u'_id': u'GRAY'}
{u'count': 216292, u'_id': u'BLACK'}
{u'count': 13160, u'_id': u'BEIGE'}


In [10]:
stopsByArrestType=datarec.aggregate([{"$group" : {"_id":"$arrest_type", "count":{"$sum":1}}}])

In [11]:
for doc in stopsByArrestType:
    print doc

{u'count': 44, u'_id': u'K - Aircraft Assist'}
{u'count': 299, u'_id': u'J - Unmarked Moving Radar (Moving)'}
{u'count': 260, u'_id': u'P - Mounted Patrol'}
{u'count': 561, u'_id': u'H - Unmarked Moving Radar (Stationary)'}
{u'count': 1545, u'_id': u'I - Marked Moving Radar (Moving)'}
{u'count': 399, u'_id': u'C - Marked VASCAR'}
{u'count': 232, u'_id': u'D - Unmarked VASCAR'}
{u'count': 6882, u'_id': u'E - Marked Stationary Radar'}
{u'count': 12875, u'_id': u'S - License Plate Recognition'}
{u'count': 1632, u'_id': u'M - Marked (Off-Duty)'}
{u'count': 5083, u'_id': u'R - Unmarked Laser'}
{u'count': 161, u'_id': u'N - Unmarked (Off-Duty)'}
{u'count': 10336, u'_id': u'L - Motorcycle'}
{u'count': 110653, u'_id': u'Q - Marked Laser'}
{u'count': 4077, u'_id': u'G - Marked Moving Radar (Stationary)'}
{u'count': 11397, u'_id': u'O - Foot Patrol'}
{u'count': 35195, u'_id': u'B - Unmarked Patrol'}
{u'count': 772, u'_id': u'F - Unmarked Stationary Radar'}
{u'count': 890886, u'_id': u'A - Marked

In [14]:
data = pd.DataFrame(list(collections.find()))

In [15]:
client.close()

In [17]:
print data.date_of_stop.dt.dayofweek.value_counts()

1    187270
2    176103
4    173063
3    168624
0    147816
5    127443
6    112970
Name: date_of_stop, dtype: int64
